# Sync open numbers

## Purpose

Add all ddf datasets published by open numbers to the ETL automatically, both as github steps and as open numbers steps.

## Find all repos

In [ ]:
import json

In [ ]:
(repos_json,) = !gh repo list -L 1000 --json url open-numbers

In [ ]:
repos = [r["url"].split("/")[-1] for r in json.loads(repos_json) if "ddf--" in r["url"]]

In [ ]:
repos

## For each repo

In [ ]:
import yaml

In [ ]:
import os

In [ ]:
from pathlib import Path

In [ ]:
from etl import paths

In [ ]:
with open(paths.DAG_FILE) as istream:
    dag = yaml.safe_load(istream)

In [ ]:
dag["steps"]

In [ ]:
import sh

In [ ]:
for repo in repos:
    assert repo.startswith("ddf--")
    short_repo = repo[5:].replace("-", "_")

    # modify the DAG
    github_step = f"github://open-numbers/{repo}"
    open_numbers_step = f"data://open_numbers/open_numbers/latest/{short_repo}"
    dag["steps"][open_numbers_step] = [github_step]

    # symlink the matching python modules
    module_path = paths.STEP_DIR / "data/open_numbers/open_numbers/latest"
    module_path.mkdir(parents=True, exist_ok=True)
    module_file = module_path / f"{short_repo}.py"

    base_file = paths.BASE_DIR / "etl/open_numbers.py"
    sh.rm("-f", module_file.as_posix())  # type: ignore
    sh.ln("-s", "../../../../open_numbers.py", module_file.name, _cwd=module_file.parent)  # type: ignore

In [ ]:
dag

## Save the DAG

In [ ]:
with open(paths.DAG_FILE.as_posix(), "w") as ostream:
    yaml.safe_dump(dag, ostream)